# 0 Prerequisities

## 0.1 Download annotation and indexes

```bash
$ wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/ensembl-96/mus_musculus.tar.gz
```

Extract the archive and use the prebuilt index for alignment-free quantification. Additionally, get the information about gene_biotype to use later

## 0.2 Software requirements

In order to execute the commands below, software listed below needs to be installed and set up properly.
- [python 3](https://www.python.org/)
- [snakemake](https://snakemake.readthedocs.io/en/stable/)
- [kallisto](https://pachterlab.github.io/kallisto/download)
- [pandas](https://pandas.pydata.org/)
- [tximport](https://bioconductor.org/packages/release/bioc/html/tximport.html)
- [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)

In [1]:
import pandas as pd
gtf = pd.read_csv("annotation/mus_musculus/Mus_musculus.GRCm38.96.gtf", sep="\t", header=None, comment="#")
gtf = gtf[gtf[2]=="gene"]
gtf[9] = gtf[8].apply(lambda x: x.split("; ")[0].split(" ")[-1][1:-1])
gtf[10] = gtf[8].apply(lambda x: x.split("; ")[-1].split(" ")[-1][1:-2])
gtf.set_index(9)[[10]].to_csv("annotation/mus_musculus/gene_biotype.txt", sep="\t", header=None)

/tmp/ipykernel_37269/337061980.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv("annotation/mus_musculus/Mus_musculus.GRCm38.96.gtf", sep="\t", header=None, comment="#")


# 1. Download RNAseq data

## 1.1 Chen et al. (2015) Hypothalamus RNAseq

> [GSE Accession: GSE66870](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE66870)

Because Chen et al. (2015) does not provide gene exon count information in their supplementary file(s), we will have to download the raw data. After downloading the raw data, it must be mapped and gene exon counts need to be extracted. What a chore!

In [2]:
# Execute in python

list_of_gsms = [f"GSM16335{n}" for n in range(81, 87)]
with open("data/GSE66870/gsms.txt", "w") as wH:
    for gsm in list_of_gsms:
        print(gsm, file=wH)

### 1.1.1 Download RAW fastq

> Execute in command line

```bash
$ cd data/GSE66870

$ cat gsms.txt | parallel -j 1 -u "ffq --ftp {} | jq -r '.[] | .url'" > ftp_links.txt

$ cat ftp_links.txt | parallel -j 1 -u 'curl -O {}'
```

### 1.1.2 Create `data/GSE66870.sampleSheet.tsv`

```
name    condition
SRR1914886      WT
SRR1914888      WT
SRR1914890      WT
SRR1914885      KO
SRR1914887      KO
SRR1914889      KO
```

## 1.2 Gabel et al. (2015) Visual cortex RNAseq

> [GSE Accession: GSE67294](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE67294)

Because Gabel et al. (2015) does not provide exon count information in their supplementary file(s), we will have to download the raw data. After downloading the raw data, it must be mapped and gene exon counts need to be extracted. What a chore!

In [3]:
# Execute in python

list_of_gsms = [f"GSM16439{n}" for n in range(40, 46)]
with open("data/GSE67294/gsms.txt", "w") as wH:
    for gsm in list_of_gsms:
        print(gsm, file=wH)

### 1.1.1 Download RAW fastq

> Execute in command line

```bash
$ cd data/GSE67294

$ cat gsms.txt | parallel -j 1 -u "ffq --ftp {} | jq -r '.[] | .url'" > ftp_links.txt

$ cat ftp_links.txt | parallel -j 1 -u 'curl -O {}'
```

### 1.1.2 Create `data/GSE67294.sampleSheet.tsv`

```
name    condition
SRR1930037      WT
SRR1930038      WT
SRR1930039      WT
SRR1930034      KO
SRR1930035      KO
SRR1930036      KO
```

## 1.3 Boxer et al. (2020) Forebrain RNAseq

> [GSE Accession: GSE128178](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE128178)

Since we are downloading raw data for Gabel et al. (2015) and Chen et al. (2015), why not do the same thing for this one too, eh! More chores just for consistency.

In [4]:
list_of_gsms = [f"GSM3666{n}" for n in range(190, 210)]
with open("data/GSE128178/gsms.txt", "w") as wH:
    for gsm in list_of_gsms:
        print(gsm, file=wH)

### 1.1.1 Download RAW fastq

> Execute in command line

```bash
$ cd data/GSE128178

$ cat gsms.txt | parallel -j 1 -u "ffq --ftp {} | jq -r '.[] | .url'" > ftp_links.txt

$ cat ftp_links.txt | parallel -j 1 -u 'curl -O {}'
```

### 1.1.2 Create `data/GSE128178.sampleSheet.tsv`

```
name    condition
SRR8714114      WT
SRR8714115      WT
SRR8714116      WT
SRR8714117      WT
SRR8714118      WT
SRR8714119      WT
SRR8714120      WT
SRR8714121      WT
SRR8714122      WT
SRR8714123      WT
SRR8714124      KO
SRR8714125      KO
SRR8714126      KO
SRR8714127      KO
SRR8714128      KO
SRR8714129      KO
SRR8714130      KO
SRR8714131      KO
SRR8714132      KO
SRR8714133      KO
```

## 1.5 Quantify mRNA abundance

In order to keep quantification across multiple data sets consistent, `Snakefile` contains the command line arguments to execute [kallisto](https://pachterlab.github.io/kallisto/) for the above raw fastq files

```bash
$ snakemake singleend pairedend -p -j 1
```

## 1.6 Perform differential expression analysis

Import transcript abundance according to this [workflow](https://bioconductor.org/packages/devel/bioc/vignettes/tximport/inst/doc/tximport.html) and perform differential expression analysis using [DESeq2](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)

```bash
$ snakemake counts intcounts deseq -p -j 1
```

## 1.7 Johnson et al. (2017) Excitatory neurons processed RNAseq

> [GSE Accession: GSE83474](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE83474)

The processed data readily provides gene exon counts.

In [5]:
# Execute in python

import pandas as pd
from tqdm.auto import tqdm
import urllib.request

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True, leave=False, miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)
        
download_url("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE83474&format=file", "data/GSE83474.tar")    

?acc=GSE83474&format=file: 0.00B [00:00, ?B/s]

### 1.4.1 Save the counts in `counts/GSE83474.tsv`

```bash
$ cd data

$ extract GSE83474.tar
```

In [6]:
import glob
files = sorted(glob.glob("data/*Male_excitatory_6w*.txt.gz"))
dfs = []
for file in files:
    df = pd.read_csv(file, sep="\t", index_col=0)[["Annotation", "Count"]]
    df = df[df["Annotation"]=="protein_coding"][["Count"]]
    df.columns = [file.split("_6w_")[-1].split("_count")[0]]
    dfs.append(df)
counts = pd.concat(dfs, axis=1).dropna()
counts.to_csv("counts/GSE83474.tsv", sep="\t")

### 1.4.2 Save the experiment design in `counts/GSE83474.design.tsv`

```
sample  condition
WT_nuclear_RNAseq_rep1  WT
WT_nuclear_RNAseq_rep2  WT
WT_nuclear_RNAseq_rep3  WT
WT_nuclear_RNAseq_rep4  WT
R106W_nuclear_RNAseq_rep1       R106W
R106W_nuclear_RNAseq_rep2       R106W
R106W_nuclear_RNAseq_rep3       R106W
R106W_nuclear_RNAseq_rep4       R106W
T158M_nuclear_RNAseq_rep1       T158M
T158M_nuclear_RNAseq_rep2       T158M
T158M_nuclear_RNAseq_rep3       T158M
T158M_nuclear_RNAseq_rep4       T158M
```

### 1.4.3 Save the following DESeq2 code in `counts/GSE83474.DESeq2.R`

```R
library(BiocParallel)
library(DESeq2)
register(MulticoreParam(4))

deseq_function <- function(counts_file, design_file, threshold, out_prefix){
    
    counts = read.csv(counts_file, sep="\t", header = TRUE, row.names = 1, check.names = FALSE)
    design = read.csv(design_file, header=TRUE, sep="\t", row.names=1)
    
    dds <- DESeqDataSetFromMatrix(countData = counts, colData = design, design = ~ condition)
    dds <- dds[rowSums(counts(dds)) > threshold,]

    # Performing DESeq2 analysis
    dds <- DESeq(dds, parallel=TRUE)    
    saveRDS(dds, file=paste(out_prefix, "dds.rds", collapse="", sep=""))
    
    rld <- rlog(dds)
    r106w_vs_wt <- results(dds, c("condition", "R106W", "WT"), independentFiltering = TRUE)
    write.table(as.data.frame(r106w_vs_wt), file=paste( out_prefix, "R106W_vs_WT.tsv", collapse = "", sep=""), quote=F, col.names=NA, sep="\t")
    
    t158m_vs_wt <- results(dds, c("condition", "T158M", "WT"), independentFiltering = TRUE)
    write.table(as.data.frame(t158m_vs_wt), file=paste( out_prefix, "T158M_vs_WT.tsv", collapse = "", sep=""), quote=F, col.names=NA, sep="\t")
    print(paste(c(counts_file, "finished")))
}

deseq_function("GSE83474.tsv", "GSE83474.design.tsv", 20, "GSE83474.DESeq2.results/")
```

### 1.4.4 Run DESeq2

```bash
$ cd counts
$ mkdir GSE83474.DESeq2.results
$ Rscript GSE83474.DESeq2.R
```